# 94_fig4_persistence_sensitivity

Figure 4: compare temporary ξ shock persistence across two commitment runs (e.g., rho_tau=0.90 vs 0.99).

In [ ]:
import os, sys, json, numpy as np, torch, matplotlib.pyplot as plt
sys.path.append("..")

from src.config import ModelParams
from src.io_utils import load_json, load_npz, load_torch, load_selected_run, find_latest_run_dir
from src.deqn import PolicyNetwork

ART = os.path.join("..","artifacts","runs")

def get_run(policy: str) -> str:
    rd = load_selected_run(ART, policy)
    if rd is None:
        rd = find_latest_run_dir(ART, policy)
    if rd is None:
        raise RuntimeError(f"No runs found for policy={policy} under {ART}")
    return rd

def _parse_dtype(s: str):
    if s is None:
        return torch.float32
    if isinstance(s, torch.dtype):
        return s
    s = str(s)
    if "float64" in s:
        return torch.float64
    if "float32" in s:
        return torch.float32
    if "bfloat16" in s:
        return torch.bfloat16
    return torch.float32

def load_params_from_run(run_dir: str, *, device="cpu"):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    p = cfg.get("params", {})
    dtype = _parse_dtype(p.get("dtype"))
    dev = device if device is not None else p.get("device","cpu")
    keep = {k:v for k,v in p.items() if k in ModelParams.__dataclass_fields__}
    keep["device"] = dev
    keep["dtype"] = dtype
    return ModelParams(**keep).to_torch()

def load_net_from_run(run_dir: str, d_in: int, d_out: int):
    cfg = load_json(os.path.join(run_dir, "config.json"))
    tc = cfg.get("train_cfg", {})
    hidden = tuple(tc.get("hidden_layers", (512,512)))
    activation = tc.get("activation", "selu")
    net = PolicyNetwork(d_in, d_out, hidden=hidden, activation=activation)
    state = load_torch(os.path.join(run_dir, "weights.pt"), map_location="cpu")
    # state is usually a plain state_dict
    if isinstance(state, dict) and "state_dict" in state:
        state = state["state_dict"]
    net.load_state_dict(state)
    net.eval()
    return net

# --- paper reporting helpers ---
ann = lambda x: 400.0*x  # annualized percent (quarterly -> annual)


In [ ]:
from src.steady_states import solve_commitment_sss_from_policy_switching
from src.experiments import DeterministicPathSpec, simulate_deterministic_path, calibrate_xi_jump_to_match_pi_impact

ROOT = os.path.join("..","artifacts","runs","commitment")

def list_runs(root):
    if not os.path.isdir(root): return []
    runs = [os.path.join(root,d) for d in os.listdir(root) if os.path.isdir(os.path.join(root,d))]
    runs.sort(key=lambda p: os.path.getmtime(p), reverse=True)
    return runs

runs = list_runs(ROOT)
RUN_A = None
RUN_B = None
if RUN_A is None or RUN_B is None:
    if len(runs) < 2:
        raise RuntimeError("Need at least two commitment runs in artifacts/runs/commitment (trained with different rho_tau).")
    RUN_A, RUN_B = runs[0], runs[1]

print("RUN_A:", RUN_A)
print("RUN_B:", RUN_B)

paramsA = load_params_from_run(RUN_A)
paramsB = load_params_from_run(RUN_B)
netA = load_net_from_run(RUN_A, 7, 13)
netB = load_net_from_run(RUN_B, 7, 13)

sssA = solve_commitment_sss_from_policy_switching(paramsA, netA)
sssB = solve_commitment_sss_from_policy_switching(paramsB, netB)

x0A = torch.tensor([[float(sssA.by_regime[0]["Delta_prev"]), float(sssA.by_regime[0]["logA"]), float(sssA.by_regime[0]["loggtilde"]), float(sssA.by_regime[0]["xi"]), 0.0,
                     float(sssA.by_regime[0]["vartheta_prev"]), float(sssA.by_regime[0]["varrho_prev"])]], dtype=torch.float32)
x0B = torch.tensor([[float(sssB.by_regime[0]["Delta_prev"]), float(sssB.by_regime[0]["logA"]), float(sssB.by_regime[0]["loggtilde"]), float(sssB.by_regime[0]["xi"]), 0.0,
                     float(sssB.by_regime[0]["vartheta_prev"]), float(sssB.by_regime[0]["varrho_prev"])]], dtype=torch.float32)

T=40
target_pi0 = 0.01

xiA = calibrate_xi_jump_to_match_pi_impact(paramsA, "commitment", netA, x0=x0A, target_pi0=target_pi0, horizon_T=1)
xiB = calibrate_xi_jump_to_match_pi_impact(paramsB, "commitment", netB, x0=x0B, target_pi0=target_pi0, horizon_T=1)

x0A2 = x0A.clone(); x0A2[:,3] += xiA
x0B2 = x0B.clone(); x0B2[:,3] += xiB

spec = DeterministicPathSpec(T=T, epsA=0.0, epsg=0.0, epst=0.0, regime_path=None)
pathA = simulate_deterministic_path(paramsA, "commitment", netA, x0=x0A2, spec=spec, compute_implied_i=True)
pathB = simulate_deterministic_path(paramsB, "commitment", netB, x0=x0B2, spec=spec, compute_implied_i=True)

t=np.arange(T+1)
plt.figure()
plt.plot(t, ann(pathA["pi"][:,0]), label=f"Run A (rho_tau={paramsA.rho_tau})")
plt.plot(t, ann(pathB["pi"][:,0]), label=f"Run B (rho_tau={paramsB.rho_tau})", linestyle="--")
plt.axhline(0, linewidth=1)
plt.title("Figure 4: π persistence (matched impact)")
plt.xlabel("t"); plt.ylabel("π"); plt.legend(); plt.show()


In [ ]:
# Paper Figure 4 strict requirement: run in the no-regime model.
# Enforce eta_bar=0 and p12=p21=0 for both runs.
def _require_no_regimes(params, label):
    if float(params.eta_bar) != 0.0 or float(params.p12) != 0.0 or float(params.p21) != 0.0:
        raise RuntimeError(
            f"{label}: Figure 4 in the paper is defined for the no-regime model only (eta_bar=0, p12=0, p21=0). "
            f"Current values: eta_bar={params.eta_bar}, p12={params.p12}, p21={params.p21}."
        )

_require_no_regimes(paramsA, 'Run A')
_require_no_regimes(paramsB, 'Run B')
